# PRE-SETTING


In [3]:
#Per impostare il locale italiano
# 1 Decommentare l'ultima riga ed eseguire la cella
# 2 Selezionare la codifica it_IT.UTF-8 '282'
# 3 Riavviare il runtime ed eseguire questa cella con l'ultima riga commentata aaaa

# Da non eseguire in Locale
!export LC_ALL="it_IT.UTF-8"
!export LC_CTYPE="it_IT.UTF-8"
#!sudo dpkg-reconfigure locales

"export" non Ã¨ riconosciuto come comando interno o esterno,
 un programma eseguibile o un file batch.
"export" non Ã¨ riconosciuto come comando interno o esterno,
 un programma eseguibile o un file batch.


In [2]:
import locale
from dask.diagnostics import ProgressBar  
locale.setlocale(locale.LC_ALL, 'it_IT.UTF-8')

'it_IT.UTF-8'

In [ ]:
!pip install aiohttp
!pip install requests
!pip install dask[dataframe] --upgrade



In [3]:
import dask.dataframe as dd
import math

#DEFINING BASE BENCH CLASS

In [29]:
class BaseDfBench(object):
    def __init__(self, df):
      self.df = df

    def load_dataset(self, path, format, conn=None, **kwargs):
        """
        Load the provided dataframe
        """
        if format == "csv":
            self.df = self.read_csv(path, **kwargs)
        elif format == "json":
            self.df = self.read_json(path, **kwargs)
        elif format == "xml":
            self.df = self.read_xml(path, **kwargs)
        elif format == "excel":
            self.df = self.read_excel(path, **kwargs)
        elif format == "parquet":
            self.df = self.read_parquet(path, **kwargs)
        elif format == "sql": 
            self.df = self.read_sql(path, conn, **kwargs)            
        return self.df        
        
    def read_sql(self, query, conn, **kwargs):
        """
        Given a connection and a query
        creates a dataframe from the query output
        """
        self.df = dd.read_sql(query, conn)
        return self.df
    def read_json(self, path, **kwargs):
        """
        Read a json file
        """
        self.df = dd.read_json(path, **kwargs)
        return self.df
    
    def read_csv(self, path, **kwargs):
        """
        Read a csv file
        """
        self.df = dd.read_csv(path, **kwargs)
        return self.df
        
    def read_xml(self, path, **kwargs):
        """
        Read a xml file
        """
        self.df = dd.read_xml(path, **kwargs)
        return self.df
        
    def read_excel(self, path, **kwargs):
        """
        Read an excel file
        """
        self.df = dd.read_excel(path, **kwargs)
        return self.df
        
    def read_parquet(self, path, **kwargs):
        """
        Read a parquet file
        """
        self.df = dd.read_parquet(path, **kwargs)
        return self.df
    def sort(self, columns, ascending=True):
        """
        Sort the dataframe by the provided columns
        Columns is a list of column names
        """
        self.df = self.df.sort_values(columns, ascending=ascending)
        return self.df

    def get_columns(self):
        """
        Return the name of the columns in the dataframe
        """
        return list(self.df.columns.values)

    def is_unique(self, column):
        """
        Check the uniqueness of all values contained in the provided column_name
        """
        return self.df[column].is_unique

    def delete_columns(self, columns):
        """
        Delete the specified columns
        Columns is a list of column names
        """
        self.df = self.df.drop(columns=columns)
        return self.df

    def rename_columns(self, columns):
        """
        Rename the provided columns using the provided names
        Columns is a dictionary: {"column_name": "new_name"}
        """
        self.df = self.df.rename(columns=columns)
        return self.df

    def merge_columns(self, columns, separator, name):
        """
        Create a new column with the provided name combining the two provided columns using the provided separator
        Columns is a list of two column names; separator and name are strings
        """
        self.df[name] = self.df[columns[0]].astype(str) + separator + self.df[columns[1]].astype(str)
        return self.df

    def fill_nan(self, value):
        """
        Fill nan values in the dataframe with the provided value
        """
        self.df = self.df.fillna(value)
        return self.df
        
    def one_hot_encoding(self, columns):
        """
        Performs one-hot encoding of the provided columns
        Columns is a list of column names
        """
        dummies = dd.get_dummies(self.df[columns])
        self.df = dd.concat([self.df.drop(columns=columns), dummies], axis=1)
        return self.df

    def locate_null_values(self, column):
        """
        Returns the rows of the dataframe which contains
        null value in the provided column.
        """
        return self.df[self.df[column].isna()]
    def search_by_pattern(self, column, pattern):
        """
        Returns the rows of the dataframe which
        match with the provided pattern
        on the provided column.
        Pattern could be a regular expression.
        """
        return self.df[self.df[column].str.contains(re.compile(pattern))]
        
    def locate_outliers(self, column, lower_quantile=0.1, upper_quantile=0.99):
        """
        Returns the rows of the dataframe that have values
        in the provided column lower or higher than the values
        of the lower/upper quantile.
        """
        q_low = self.df[column].quantile(lower_quantile)
        q_hi  = self.df[column].quantile(upper_quantile)
        return self.df[(self.df[column] < q_low) | (self.df[column] > q_hi)]
        
    def get_columns_types(self):
        """
        Returns a dictionary with column types
        """
        return self.df.dtypes.apply(lambda x: x.name).to_dict()
        
    def cast_columns_types(self, converterList: list, str_date_time_format = '%d %B %Y'):
        """
        Cast the data types of the provided columns 
        to the provided new data types.
        dtypes is a dictionary that provide for each
        column to cast the new data type.
        """
        columnsDate = []
        for item in converterList:
            if(item['correct_dtype'] == float):
              self.df[item['col']] = self.df[item['col']].str.replace('.', '').str.replace(',', '.').str.extract(r'([+-]?[0-9]+\.[0-9]+)', expand=False).astype(float)
            elif(item['correct_dtype'] == 'datetime'):
              columnsDate.append(item['col'])
        
        self.change_date_time_format(columnsDate, str_date_time_format = '%d %B %Y')
        return self.df
        
        
    def get_stats(self):
        """
        Returns dataframe statistics.
        Only for numeric columns.
        Min value, max value, average value, standard deviation, and standard quantiles.
        """
        return self.df.describe()
        
    
    def assign_custom_types(self, mismatched_types, correct_types):

      for mismatch in mismatched_types:
        mismatch['correct_dtype'] = correct_types[mismatch['col']]
      
      return mismatched_types


        
    def find_mismatched_dtypes(self):
      """
      L'implementazione originaria non si adattava alla nostra libreria.
      Dask interpreta i mismatch nella colonne come object, per cui facendo un rapido controllo 
      su questo tipo di dato, otteniamo le colonne che dobbiamo andare a sistemare.
      Il risultato è un Set con due liste che riportano rispettivamente i nomi delle colonne con type=object
      e la seconda lista i rispettivi indici.
      """
      current_dtypes = self.get_columns_types()

      out = []
      for k in current_dtypes.keys():
          if current_dtypes[k] == 'object':
            out.append({'col': k, 'current_dtype': current_dtypes[k], 'correct_dtype': ''})
      return out
        
    def check_allowed_char(self, column, pattern):
        """
        Return true if all the values of the provided column
        follow the provided pattern.
        For example, if the pattern [a-z] is provided the string
        'ciao' will return true, the string 'ciao123' will return false.
        """
        return self.df[column].str.contains(re.compile(pattern)).all()
        
    def drop_duplicates(self):
        """
        Drop duplicate rows.
        """
        self.df = self.df.drop_duplicates()
        return self.df
        
    def drop_by_pattern(self, column, pattern):
        """
        Delete the rows where the provided pattern
        occurs in the provided column.
        """
        matching_rows = self.search_by_pattern(column, pattern)
        self.df = self.df.drop(matching_rows.index)
        return self.df
        
    def change_date_time_format(self, columns: list, str_date_time_format):
        """
        Change the date/time format of the provided column
        according to the provided formatting string.
        column datatype must be datetime
        An example of str_date_time_format is '%m/%d/%Y'
        """
        for col in columns:
          self.df[col] = dd.to_datetime(self.df[col], format=str_date_time_format, dayfirst=True, errors='coerce')
        return self.df
        
    def set_header_case(self, case):
        """
        Put dataframe headers in the provided case
        Supported cases: "lower", "upper", "title", "capitalize", "swapcase"
        (see definitions in pandas documentation)
        """
        if mode == "lower":
            self.df.columns = map(str.lower, self.df.columns)
        elif mode == "upper":
            self.df.columns = map(str.upper, self.df.columns)
        elif mode == "title":
            self.df.columns = map(str.title, self.df.columns)
        elif mode == "capitalize":
            self.df.columns = map(str.capitalize, self.df.columns)
        elif mode == "swapcase":
            self.df.columns = map(str.swapcase, self.df.columns)
        return self.df

    def set_content_case(self, columns, case):
        """
        Put dataframe content in the provided case
        Supported cases: "lower", "upper", "title", "capitalize", "swapcase"
        (see definitions in pandas documentation)
        Columns is a list of two column names; empty list for the whole dataframe
        """
        if len(columns) == 0:
            columns = list(self.df.columns.values)
        for column in columns:
            if mode == "lower":
                self.df[column] = self.df[column].str.lower()
            elif mode == "upper":
                self.df[column] = self.df[column].str.upper()
            elif mode == "title":
                self.df[column] = self.df[column].str.title()
            elif mode == "capitalize":
                self.df[column] = self.df[column].str.capitalize()
            elif mode == "swapcase":
                self.df[column] = self.df[column].str.swapcase()
        return self.df

    def duplicate_columns(self, columns):
        """
        Duplicate the provided columns (add to the dataframe with "_duplicate" suffix)
        Columns is a list of column names
        """
        for column in columns:
            self.df[column + "_duplicate"] = self.df[column]
        return self.df

    def pivot(self, index, columns, values, aggfunc):
        """
        Define the lists of columns to be used as index, columns and values respectively,
        and the dictionary to aggregate ("sum", "mean", "count") the values for each column: {"col1": "sum"}
        (see pivot_table in pandas documentation)
        """
        self.df = dd.pivot_table(self.df, index=index, values=values, columns=columns, aggfunc=aggfunc).reset_index()
        return self.df

    def unpivot(self, columns, var_name, val_name):
        """
        Define the list of columns to be used as values for the variable column,
        the name for variable columns and the one for value column_name
        """
        self.df = dd.melt(self.df, id_vars=list(set(list(self.df.columns.values)) - set(columns)), value_vars=columns, var_name=var_name, value_name=val_name)
        return self.df

    def delete_empty_rows(self, columns):
        """
        Delete the rows with null values for all provided Columns
        Columns is a list of column names
        """
        self.df = self.df.dropna(subset = columns, inplace=True)
        return self.df

    def split(self, column, sep, splits, col_names):
        """
        Split the provided column into splits + 1 columns named after col_names
        using the provided sep string as separator
        Col_names is a list of column names
        """
        self.df[col_names] = self.df[column].str.split(sep, splits, expand=True)
        return self.df

    def strip(self, columns, chars):
        """
        Remove the characters appearing in chars at the beginning/end of the provided columns
        Columns is a list of column names
        """
        for column in columns:
            self.df[column] = self.df[column].str.strip(chars)
        return self.df

    def remove_diacritics(self, columns):
        """
        Remove diacritics from the provided columns
        Columns is a list of column names
        """
        for column in columns:
            self.df[column] = self.df[column].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')
        return self.df
        
    def set_index(self, column):
        """
        Set the provided column as index
        """
        self.df = self.df.set_index(column)
        return self.df
        
        
    def change_num_format(self, formats):
        """
        Round one ore more columns to a variable number of decimal places.
        formats is a dictionary with the column names as key and the number of decimal places as value.
        """
        self.df = self.df.round(formats)
        return self.df
        
        
    def calc_column(self, col_name, f):
        """
        Calculate the new column col_name by applying
        the function f
        """
        self.df[col_name] = self.df.apply(f, axis=1)
        return self.df
        
    def join(self, other, left_on=None, right_on=None, how='inner', **kwargs):
        """
        Joins current dataframe (left) with a new one (right).
        left_on/right_on are the keys on which perform the equijoin
        how is the type of join
        **kwargs: additional parameters
        
        The result is stored in the current dataframe.
        """
        self.df = self.df.merge(other, left_on=left_on, right_on=right_on, how=how, **kwargs)
        return self.df
        
    def groupby(self, columns, f):
        """
        Aggregate the dataframe by the provided columns
        then applied the function f on every group
        """
        return self.df.groupby(columns).agg(f)
        
    
    def categorical_encoding(self, columns):
        """
        Convert the categorical values in these columns into numerical values
        Columns is a list of column names
        """
        for column in columns:
            self.df[column] = self.df[column].astype('category')
            self.df[column] = self.df[column].cat.codes
        return self.df

    def sample_rows(self, frac, num):
        """
        Return a sample of the rows of the dataframe
        Frac is a boolean:
        - if true, num is the percentage of rows to be returned
        - if false, num is the exact number of rows to be returned
        """
        if frac:
            return self.df.sample(frac=num/100)
        else:
            return self.df.sample(n=num)

    def append(self, other, ignore_index):
        """
        Append the rows of another dataframe (other) at the end of the provided dataframe
        All columns are kept, eventually filled by nan
        Ignore index is a boolean: if true, reset row indices
        """
        self.df = self.df.append(other, ignore_index=ignore_index)
        return self.df

    def replace(self, columns, to_replace, value, regex):
        """
        Replace all occurrencies of to_replace (numeric, string, regex, list, dict) in the provided columns using the provided value
        Regex is a boolean: if true, to_replace is interpreted as a regex
        Columns is a list of column names
        """
        self.df[columns] = self.df[columns].replace(to_replace=to_replace, value=value, regex=regex)
        return self.df

    def edit(self, columns, func):
        """
        Edit the values of the cells in the provided columns using the provided expression
        Columns is a list of column names
        """
        self.df[columns] = self.df[columns].apply(func)
        return self.df

    def set_value(self, index, column, value):
        """
        Set the cell identified by index and column to the provided value
        """
        self.df.at[index, column] = value
        return self.df

    def min_max_scaling(self, columns):
        """
        Independently scale the values in each provided column in the range (0, 1)
        Columns is a list of column names
        """
        for column in columns:
            self.df[column] = self.df[column] - self.df[column].min()
            self.df[column] = self.df[column] / self.df[column].max()
            self.df[column] = self.df[column] * (max - min) + min
        return self.df

    def define_min_max(self, columns):
        for column in columns:
          if (self.df[column].min())

    def round(self, columns, n):
        """
        Round the values in columns using n decimal places
        Columns is a list of column names
        """
        self.df[columns] = self.df[columns].round(n)
        return self.df
        
    def get_duplicate_columns(self):
        """
        Return a list of duplicate columns, if exists.
        Duplicate columns are those which have same values for each row.
        """
        cols = self.df.columns.values
        return [(cols[i], cols[j]) for i in range(0, len(cols)) for j in range(i+1, len(cols)) if self.df[cols[i]].equals(self.df[cols[j]])]
    
    def to_csv(self, path, **kwargs):
        """
        Export the dataframe in a csv file.
        """
        self.df.to_csv(path, **kwargs)
        pass
        
    def query(self, query):
        """
        Queries the dataframe and returns the corresponding
        result set.
        :param query: a string with the query conditions, e.g. "col1 > 1 & col2 < 10"
        :return: subset of the dataframe that correspond to the selection conditions
        """
        return self.df.query(query)
    def missing_values_percent(self):
      return self.df


# EXTRACT CSV FILE

In [ ]:
from pathlib import Path
import pandas as pd


path = 'https://dbgroup.ing.unimore.it/invoices/data.zip'
!wget -nc 'https://dbgroup.ing.unimore.it/invoices/data.zip'
!unzip '/content/data.zip'


#dtype={'billing_frequency': 'string','gas_offer': 'float64'}

--2022-01-29 15:00:15--  https://dbgroup.ing.unimore.it/invoices/data.zip
Resolving dbgroup.ing.unimore.it (dbgroup.ing.unimore.it)... 155.185.48.139
Connecting to dbgroup.ing.unimore.it (dbgroup.ing.unimore.it)|155.185.48.139|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1672160143 (1.6G) [application/zip]
Saving to: 'data.zip'

data.zip            100%[===================>]   1.56G  23.4MB/s    in 69s     

2022-01-29 15:01:24 (23.2 MB/s) - 'data.zip' saved [1672160143/1672160143]

Archive:  /content/data.zip
  inflating: invoices.csv            


# SETTING RIGHT TYPE BY DOCUMENTATION

In [15]:
correctTypes= {
    'F1_kWh': float,
    'F2_kWh': float,
    'F3_kWh': float,
    'average_gas_bill_cost': float,
    'average_light_bill_cost': float,
    'average_unit_gas_cost': float,
    'average_unit_light_cost': float,
    'bill_id': int ,
    'bill_type': str,
    'billing_frequency': str,
    'date': 'datetime',
    'emission_date': 'datetime',
    'extra_fees': float,
    'gas_amount': float,
    'gas_average_cost': float,
    'gas_consumption': float,
    'gas_end_date': 'datetime',
    'gas_material_cost': float,
    'gas_offer': str, #float su pdf
    'gas_start_date': 'datetime',
    'gas_system_charges': float,
    'gas_transport_cost': float,
    'howmuch_pay': float,
    'light_amount': float,
    'light_average_cost': float,
    'light_consumption': float,
    'light_end_date': 'datetime',
    'light_material_cost': float,
    'light_offer': str,
    'light_offer_type': str,
    'light_start_date': 'datetime',
    'light_system_charges': float,
    'light_transport_cost': float,
    'supply_type': str,
    'total_amount': float,
    'tv': float,
}

# PIPELINE


## 1) DATA INGESTION AND DISCOVERY

### 1.1 Read data from source

In [58]:
#Decomentare la prima o la seconda riga in base al tipo di enviroment 

#general = dd.read_csv('/content/invoices.csv', dtype={'billing_frequency': 'string', 'gas_offer': 'string', 'city':'string'}, low_memory=False) #online
general = dd.read_csv('invoices.csv', dtype={'billing_frequency': 'string', 'gas_offer': 'float64', 'city':'string'}, low_memory=False) #locale
utilities = general[['user_code', 'customer_code', 'city', 'address']]
customers = general[['user_code', 'nominative', 'sex', 'age']]
invoices = general.drop(labels=['user_code', 'customer_code', 'city', 'address', 'nominative', 'sex', 'age'], axis=1)
#invoicesNotDerived = invoices.drop(labels=['total_amount', 'howmuch_pay', 'light'], axis=1)
myBase = BaseDfBench(invoices)
myBase.df.
#(14,15,29,30,31,32,33,36) mixed type

Dask Series Structure:
npartitions=79
    bool
     ...
    ... 
     ...
     ...
Name: bill_type, dtype: bool
Dask Name: getitem, 237 tasks

In [13]:
myBase.df.head()

,bill_id,F1_kWh,F2_kWh,F3_kWh,date,light_start_date,light_end_date,tv,gas_amount,gas_average_cost,...,average_unit_gas_cost,average_gas_bill_cost,billing_frequency,bill_type,gas_system_charges,light_system_charges,gas_material_cost,light_transport_cost,gas_transport_cost,light_material_cost
0,0,0.0,0.0,0.0,16 Aprile 2018,16 Aprile 2018,25 Novembre 2019,"0,00","-0,48",NaN,...,NaN,NaN,<NA>,True,-0.06,NaN,0.06,NaN,-0.20,NaN
1,1,81.0,62.0,76.0,5 Dicembre 2020,5 Dicembre 2020,31 Dicembre 2020,"0,00",NaN,NaN,...,NaN,NaN,<NA>,True,NaN,9.15,NaN,5.68,NaN,23.13
2,2,0.0,0.0,0.0,5 Dicembre 2020,5 Dicembre 2020,31 Dicembre 2020,"0,00","-21,79",NaN,...,NaN,NaN,<NA>,True,1.24,NaN,-10.13,NaN,-7.13,NaN
3,3,0.0,0.0,0.0,3 Ottobre 2020,3 Ottobre 2020,31 Dicembre 2020,"0,00","58,51","0,44 €/smc",...,"0,44 €/smc","1,08 €/smc",<NA>,True,-0.88,NaN,23.68,NaN,17.66,NaN
4,4,0.0,0.0,0.0,16 Dicembre 2020,16 Dicembre 2020,31 Dicembre 2020,"0,00","383,66","0,32 €/smc",...,"0,32 €/smc","0,86 €/smc",<NA>,True,14.71,NaN,141.57,NaN,63.59,NaN


In [65]:
myBase.df[myBase.df['bill_type'] == True]['bill_type'].unique().compute()

0    True
Name: bill_type, dtype: bool

### 1.2 Study how to deal with data that doesn’t fit in memory (!)

### 1.4 Locate outliers

### 1.3 Locate missing values


In [ ]:
from dask.diagnostics import ProgressBar  
missing_values = myBase.df.isna().sum()

with ProgressBar():
  percent_missing = ((missing_values / myBase.df.index.size) * 100).compute()
percent_missing

[########################################] | 100% Completed |  1min 47.1s


bill_id                     0.000000
F1_kWh                      0.001581
F2_kWh                      0.000876
F3_kWh                      0.006564
date                        0.074982
light_start_date            0.074982
light_end_date              0.074982
tv                          0.000000
gas_amount                 35.121499
gas_average_cost           40.844314
light_average_cost         56.848449
emission_date               0.000000
supply_type                 0.000000
gas_start_date              0.074982
gas_end_date                0.074982
extra_fees                  0.000000
gas_consumption            35.121490
light_consumption          54.174903
gas_offer                   0.000000
light_offer_type            0.000000
light_offer                 0.000000
howmuch_pay                 0.000000
total_amount                0.000000
light_amount               54.174779
average_unit_light_cost    56.847982
average_light_bill_cost    56.845172
average_unit_gas_cost      40.844314
a

## 2) DATA VALIDATION


### 2.3 Find mismatched types

In [66]:
mismatchedTypes = myBase.find_mismatched_dtypes()
mismatchedTypesCorrect = myBase.assign_custom_types(mismatchedTypes, correctTypes)
mismatchedTypesCorrect
myBase.df = myBase.cast_columns_types(mismatchedTypesCorrect)
myBase.df


C:\Users\legol\AppData\Local\Programs\Python\Python39\lib\site-packages\dask\dataframe\accessor.py:49: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  out = getattr(getattr(obj, accessor, obj), attr)(*args, **kwargs)


,bill_id,F1_kWh,F2_kWh,F3_kWh,date,light_start_date,light_end_date,tv,gas_amount,gas_average_cost,light_average_cost,emission_date,supply_type,gas_start_date,gas_end_date,extra_fees,gas_consumption,light_consumption,gas_offer,light_offer_type,light_offer,howmuch_pay,total_amount,light_amount,average_unit_light_cost,average_light_bill_cost,average_unit_gas_cost,average_gas_bill_cost,billing_frequency,bill_type,gas_system_charges,light_system_charges,gas_material_cost,light_transport_cost,gas_transport_cost,light_material_cost
npartitions=79,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
,int64,float64,float64,float64,datetime64[ns],datetime64[ns],datetime64[ns],float64,float64,float64,float64,datetime64[ns],object,datetime64[ns],datetime64[ns],float64,float64,float64,float64,object,uint64,float64,float64,float64,float64,float64,float64,float64,string,bool,float64,float64,float64,float64,float64,float64
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


### 2.1 Check data range

In [70]:
with ProgressBar():
  prova = myBase.df.describe().compute()
prova

[                                        ] | 0% Completed |  5.5s

C:\Users\legol\AppData\Local\Programs\Python\Python39\lib\site-packages\dask\dataframe\accessor.py:49: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  out = getattr(getattr(obj, accessor, obj), attr)(*args, **kwargs)


[########################################] | 100% Completed |  5min 16.3s


,bill_id,F1_kWh,F2_kWh,F3_kWh,tv,gas_amount,gas_average_cost,light_average_cost,extra_fees,gas_consumption,...,average_unit_light_cost,average_light_bill_cost,average_unit_gas_cost,average_gas_bill_cost,gas_system_charges,light_system_charges,gas_material_cost,light_transport_cost,gas_transport_cost,light_material_cost
count,1.049714e+07,1.049698e+07,1.049705e+07,1.049645e+07,1.049714e+07,6.810389e+06,6.209657e+06,4.529680e+06,1.049714e+07,6.810390e+06,...,4.529729e+06,4.530024e+06,6.209657e+06,6.211363e+06,6.734656e+06,4.755502e+06,6.741981e+06,4.756329e+06,6.735368e+06,4.756322e+06
mean,1.043021e+06,5.909597e+01,4.892921e+01,5.317689e+01,5.308814e+00,1.591596e+02,7.418461e-01,4.880660e+01,4.326666e+00,1.643498e+02,...,2.953825e-01,7.787436e-01,7.418461e-01,1.552363e+00,4.785760e+00,2.027110e+01,6.399663e+01,1.729069e+01,3.748406e+01,4.869274e+01
std,6.862310e+05,1.285331e+03,9.459065e+02,1.475954e+03,7.998176e+00,3.541522e+02,1.934005e+00,1.657033e+01,1.802168e+02,1.869755e+02,...,1.771076e+00,4.856549e+00,1.934005e+00,3.695770e+00,1.946054e+01,5.062049e+01,1.235472e+02,3.590962e+01,6.705752e+01,8.804623e+01
min,0.000000e+00,-9.923490e+05,-9.900540e+05,-9.873120e+05,-4.780000e+02,-6.610243e+04,0.000000e+00,0.000000e+00,-8.609044e+04,-9.990000e+02,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,-3.449810e+03,-2.181506e+04,-1.891876e+04,-2.001829e+04,-9.773770e+03,-1.959404e+04
25%,4.678870e+05,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,7.487000e+01,3.300000e-01,4.600000e+01,0.000000e+00,6.700000e+01,...,1.200000e-01,2.400000e-01,3.300000e-01,8.000000e-01,-9.000000e-01,1.175000e+01,3.250000e+01,1.511000e+01,2.259000e+01,2.996000e+01
50%,9.944888e+05,2.200000e+01,1.700000e+01,2.600000e+01,0.000000e+00,1.362900e+02,4.400000e-01,5.200000e+01,5.500000e-01,1.550000e+02,...,1.500000e-01,2.900000e-01,4.400000e-01,9.700000e-01,4.700000e+00,1.932000e+01,5.656000e+01,1.746000e+01,3.488000e+01,4.548000e+01
75%,1.572120e+06,1.410000e+02,1.260000e+02,1.580000e+02,1.800000e+01,3.367000e+02,7.900000e-01,5.600000e+01,6.900000e-01,4.440000e+02,...,1.900000e-01,3.800000e-01,7.900000e-01,1.580000e+00,1.901000e+01,2.897000e+01,1.354800e+02,2.433000e+01,6.378000e+01,6.850000e+01
max,2.531054e+06,2.141161e+06,1.285834e+06,2.552774e+06,1.980000e+02,3.871433e+05,8.804400e+02,6.785000e+03,1.280832e+05,9.990000e+02,...,2.691670e+03,5.958330e+03,8.804400e+02,2.379340e+03,1.684344e+04,3.877083e+04,1.225425e+05,2.998569e+04,5.435624e+04,6.190319e+04


In [76]:
import pandas as pd 
pd.set_option('display.float_format', lambda x: '%.3f' % x)

prova

,bill_id,F1_kWh,F2_kWh,F3_kWh,tv,gas_amount,gas_average_cost,light_average_cost,extra_fees,gas_consumption,...,average_unit_light_cost,average_light_bill_cost,average_unit_gas_cost,average_gas_bill_cost,gas_system_charges,light_system_charges,gas_material_cost,light_transport_cost,gas_transport_cost,light_material_cost
count,10497143.000,10496977.000,10497051.000,10496454.000,10497143.000,6810389.000,6209657.000,4529680.000,10497143.000,6810390.000,...,4529729.000,4530024.000,6209657.000,6211363.000,6734656.000,4755502.000,6741981.000,4756329.000,6735368.000,4756322.000
mean,1043020.904,59.096,48.929,53.177,5.309,159.160,0.742,48.807,4.327,164.350,...,0.295,0.779,0.742,1.552,4.786,20.271,63.997,17.291,37.484,48.693
std,686230.982,1285.331,945.907,1475.954,7.998,354.152,1.934,16.570,180.217,186.975,...,1.771,4.857,1.934,3.696,19.461,50.620,123.547,35.910,67.058,88.046
min,0.000,-992349.000,-990054.000,-987312.000,-478.000,-66102.430,0.000,0.000,-86090.440,-999.000,...,0.000,0.000,0.000,0.000,-3449.810,-21815.060,-18918.760,-20018.290,-9773.770,-19594.040
25%,467887.000,0.000,0.000,0.000,0.000,74.870,0.330,46.000,0.000,67.000,...,0.120,0.240,0.330,0.800,-0.900,11.750,32.500,15.110,22.590,29.960
50%,994488.750,22.000,17.000,26.000,0.000,136.290,0.440,52.000,0.550,155.000,...,0.150,0.290,0.440,0.970,4.700,19.320,56.560,17.460,34.880,45.480
75%,1572119.500,141.000,126.000,158.000,18.000,336.700,0.790,56.000,0.690,444.000,...,0.190,0.380,0.790,1.580,19.010,28.970,135.480,24.330,63.780,68.500
max,2531054.000,2141161.000,1285834.000,2552774.000,198.000,387143.350,880.440,6785.000,128083.190,999.000,...,2691.670,5958.330,880.440,2379.340,16843.440,38770.830,122542.520,29985.690,54356.240,61903.190


In [114]:
with ProgressBar():
  cloneDf = myBase.df
  cloneDf = cloneDf.compute().sort_values('gas_amount', ascending=False)

cloneDf.head(1)

[########################################] | 100% Completed |  5min 54.9s


,bill_id,F1_kWh,F2_kWh,F3_kWh,date,light_start_date,light_end_date,tv,gas_amount,gas_average_cost,...,average_unit_gas_cost,average_gas_bill_cost,billing_frequency,bill_type,gas_system_charges,light_system_charges,gas_material_cost,light_transport_cost,gas_transport_cost,light_material_cost
24870,651123,0.000,0.000,0.000,2018-11-02,2018-11-02,2021-05-21,0.000,387143.350,0.210,...,0.210,0.680,<NA>,True,16843.440,NaN,122542.520,NaN,54356.240,NaN


In [ ]:
cloneDf.tail(1)

In [112]:
myBase.df['gas_amount'].head()

C:\Users\legol\AppData\Local\Programs\Python\Python39\lib\site-packages\dask\dataframe\accessor.py:49: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  out = getattr(getattr(obj, accessor, obj), attr)(*args, **kwargs)


0    -0.480
1       NaN
2   -21.790
3    58.510
4   383.660
Name: gas_amount, dtype: float64

In [109]:
myBase.df.index.values

dask.array<values, shape=(nan,), dtype=int64, chunksize=(nan,), chunktype=numpy.ndarray>

## 3) DATA STRUCTURING


### 3.3 Delete, split or merge columns

In [30]:
invoicesGas = invoices[invoices['supply_type']=='gas']

### invoicesLight #Lisa
#...
invoicesGas.head()

C:\Users\legol\AppData\Local\Programs\Python\Python39\lib\site-packages\dask\dataframe\accessor.py:49: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  out = getattr(getattr(obj, accessor, obj), attr)(*args, **kwargs)


,bill_id,F1_kWh,F2_kWh,F3_kWh,date,light_start_date,light_end_date,tv,gas_amount,gas_average_cost,...,average_unit_gas_cost,average_gas_bill_cost,billing_frequency,bill_type,gas_system_charges,light_system_charges,gas_material_cost,light_transport_cost,gas_transport_cost,light_material_cost
0,0,0.0,0.0,0.0,2018-04-16,2018-04-16,2019-11-25,0.0,-0.48,NaN,...,NaN,NaN,<NA>,True,-0.06,NaN,0.06,NaN,-0.20,NaN
3,3,0.0,0.0,0.0,2020-10-03,2020-10-03,2020-12-31,0.0,58.51,0.44,...,0.44,1.08,<NA>,True,-0.88,NaN,23.68,NaN,17.66,NaN
4,4,0.0,0.0,0.0,2020-12-16,2020-12-16,2020-12-31,0.0,383.66,0.32,...,0.32,0.86,<NA>,True,14.71,NaN,141.57,NaN,63.59,NaN
5,5,0.0,0.0,0.0,2020-11-21,2020-11-21,2020-12-31,0.0,386.63,0.37,...,0.37,0.86,<NA>,True,16.97,NaN,167.54,NaN,44.45,NaN
16,16,0.0,0.0,0.0,2019-03-22,2019-03-22,2019-03-25,0.0,12.84,0.29,...,0.29,0.48,<NA>,True,0.53,NaN,7.84,NaN,1.51,NaN


In [ ]:
# INVOICES GAS SORTED

with ProgressBar():
  invoicesGasSorted = invoicesGas.compute().sort_values('bill_id')
invoicesGasSorted

[########################################] | 100% Completed |  8min 15.8s


,bill_id,F1_kWh,F2_kWh,F3_kWh,date,light_start_date,light_end_date,tv,gas_amount,gas_average_cost,light_average_cost,emission_date,supply_type,gas_start_date,gas_end_date,extra_fees,gas_consumption,light_consumption,gas_offer,light_offer_type,light_offer,howmuch_pay,total_amount,light_amount,average_unit_light_cost,average_light_bill_cost,average_unit_gas_cost,average_gas_bill_cost,billing_frequency,bill_type,gas_system_charges,light_system_charges,gas_material_cost,light_transport_cost,gas_transport_cost,light_material_cost
0,0,0.0,0.0,0.0,2018-04-16,2018-04-16,2019-11-25,0.0,-0.48,NaN,NaN,2020-12-31,gas,2019-11-25,2018-04-16,0.48,-2.0,NaN,4255330384700204909,light,18446744073709551615,0.00,0.00,NaN,NaN,NaN,NaN,NaN,<NA>,True,-0.06,NaN,0.06,NaN,-0.20,NaN
88001,0,0.0,0.0,0.0,2021-02-03,2021-02-03,2021-03-30,0.0,11.49,NaN,NaN,2021-03-30,gas,2021-03-30,2021-02-03,0.55,-3.0,NaN,4255330384700204909,light,18446744073709551615,0.00,12.04,NaN,NaN,NaN,NaN,NaN,bimester,False,-2.26,NaN,6.24,NaN,5.53,NaN
88002,1,0.0,0.0,0.0,2021-02-03,2021-02-03,2021-03-30,0.0,217.24,0.45,NaN,2021-03-30,gas,2021-03-30,2021-02-03,0.55,243.0,NaN,17683241029697148327,light,18446744073709551615,217.79,217.79,NaN,NaN,NaN,0.45,0.89,bimester,False,14.13,NaN,109.18,NaN,30.28,NaN
88003,2,0.0,0.0,0.0,2021-02-06,2021-02-06,2021-03-30,0.0,32.33,NaN,NaN,2021-03-30,gas,2021-03-30,2021-02-06,0.00,0.0,NaN,16421179386027010504,light,18446744073709551615,32.33,32.33,NaN,NaN,NaN,NaN,NaN,<NA>,False,-0.64,NaN,4.73,NaN,22.41,NaN
84868,3,0.0,0.0,0.0,2021-04-14,2021-04-14,2021-05-27,0.0,168.03,0.23,NaN,2021-05-27,gas,2021-05-27,2021-04-14,0.00,222.0,NaN,3354194450249482832,light,18446744073709551615,168.03,168.03,NaN,NaN,NaN,0.23,0.76,bimester,True,6.34,NaN,50.30,NaN,37.61,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86501,2529555,0.0,0.0,0.0,2021-02-01,2021-02-01,2021-03-26,0.0,72.90,0.46,NaN,2021-03-26,gas,2021-03-26,2021-02-01,30.00,90.0,NaN,4426041636325184338,light,18446744073709551615,102.90,102.90,NaN,NaN,NaN,0.46,0.81,quarterly,False,-1.63,NaN,41.17,NaN,17.49,NaN
86502,2529556,0.0,0.0,0.0,2020-12-04,2020-12-04,2021-03-26,0.0,176.87,0.46,NaN,2021-03-26,gas,2021-03-26,2020-12-04,0.63,204.0,NaN,15614206351661483855,light,18446744073709551615,177.50,177.50,NaN,NaN,NaN,0.46,0.87,quarterly,False,0.80,NaN,92.98,NaN,34.63,NaN
86503,2529557,0.0,0.0,0.0,2020-12-04,2020-12-04,2021-03-26,0.0,160.68,0.42,NaN,2021-03-26,gas,2021-03-26,2020-12-04,0.80,198.0,NaN,15614206351661483855,light,18446744073709551615,161.48,161.48,NaN,NaN,NaN,0.42,0.81,quarterly,False,1.30,NaN,82.70,NaN,34.18,NaN
86504,2529558,0.0,0.0,0.0,2020-12-04,2020-12-04,2021-03-26,0.0,118.41,0.55,NaN,2021-03-26,gas,2021-03-26,2020-12-04,0.10,120.0,NaN,4426041636325184338,light,18446744073709551615,118.51,118.51,NaN,NaN,NaN,0.55,0.99,quarterly,False,-1.92,NaN,65.81,NaN,25.82,NaN


In [43]:
myBaseGas = BaseDfBench(invoicesGas)
myBaseGas.df = myBaseGas.df.drop(labels=['light_average_cost', 'light_consumption', 'light_amount',
       'average_unit_light_cost', 'average_light_bill_cost',
       'light_system_charges', 'light_transport_cost', 'light_material_cost', 'light_offer',
       'light_start_date', 'light_end_date', 'light_offer_type', 'supply_type', 'F1_kWh',
       'F2_kWh', 'F3_kWh', 'howmuch_pay'], axis=1)
myBaseGas.df.head(5)

C:\Users\legol\AppData\Local\Programs\Python\Python39\lib\site-packages\dask\dataframe\accessor.py:49: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  out = getattr(getattr(obj, accessor, obj), attr)(*args, **kwargs)


,bill_id,date,tv,gas_amount,gas_average_cost,emission_date,gas_start_date,gas_end_date,extra_fees,gas_consumption,gas_offer,total_amount,average_unit_gas_cost,average_gas_bill_cost,billing_frequency,bill_type,gas_system_charges,gas_material_cost,gas_transport_cost
0,0,2018-04-16,0.0,-0.48,NaN,2020-12-31,2019-11-25,2018-04-16,0.48,-2.0,4.255330e+18,0.00,NaN,NaN,<NA>,True,-0.06,0.06,-0.20
3,3,2020-10-03,0.0,58.51,0.44,2020-12-31,2020-12-31,2020-10-03,0.02,54.0,1.057045e+19,58.53,0.44,1.08,<NA>,True,-0.88,23.68,17.66
4,4,2020-12-16,0.0,383.66,0.32,2020-12-31,2020-12-31,2020-12-16,0.00,447.0,1.487917e+18,383.66,0.32,0.86,<NA>,True,14.71,141.57,63.59
5,5,2020-11-21,0.0,386.63,0.37,2020-12-31,2020-12-31,2020-11-21,-188.86,448.0,8.239675e+18,197.77,0.37,0.86,<NA>,True,16.97,167.54,44.45
16,16,2019-03-22,0.0,12.84,0.29,2020-12-31,2019-03-25,2019-03-22,0.00,27.0,1.487917e+18,12.84,0.29,0.48,<NA>,True,0.53,7.84,1.51


In [45]:
from dask.diagnostics import ProgressBar  
missing_values = myBaseGas.df.isna().sum()

with ProgressBar():
  percent_missing = ((missing_values / myBaseGas.df.index.size) * 100).compute()
percent_missing

[                                        ] | 0% Completed |  5.5s

C:\Users\legol\AppData\Local\Programs\Python\Python39\lib\site-packages\dask\dataframe\accessor.py:49: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  out = getattr(getattr(obj, accessor, obj), attr)(*args, **kwargs)


[########################################] | 100% Completed |  5min  1.0s


bill_id                  0.000000
date                     0.089127
tv                       0.000000
gas_amount               0.000000
gas_average_cost         8.591053
emission_date            0.000000
gas_start_date           0.089127
gas_end_date             0.089127
extra_fees               0.000000
gas_consumption          0.000000
gas_offer                0.000000
total_amount             0.000000
average_unit_gas_cost    8.591053
average_gas_bill_cost    8.577107
billing_frequency        7.806147
bill_type                0.000000
gas_system_charges       0.664370
gas_material_cost        0.572447
gas_transport_cost       0.653168
dtype: float64

In [55]:
provaGas = myBaseGas.df[(myBaseGas.df['average_unit_gas_cost'] != myBaseGas.df['gas_average_cost']) & myBaseGas.df['gas_average_cost'].isna() == False].head(10)

C:\Users\legol\AppData\Local\Programs\Python\Python39\lib\site-packages\dask\dataframe\accessor.py:49: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  out = getattr(getattr(obj, accessor, obj), attr)(*args, **kwargs)


In [56]:
provaGasComputed = provaGas[provaGas['gas_average_cost'].notna()][['average_gas_bill_cost', 'gas_amount', 'gas_consumption', 'average_unit_gas_cost', 'gas_system_charges', 'gas_transport_cost', 'gas_material_cost', 'extra_fees']]
#provaGas['check'] = provaGas[(myBaseGas.df['gas_system_charges'] + provaGas['gas_transport_cost']) == provaGas['gas_amount']]
provaGasComputed

,average_gas_bill_cost,gas_amount,gas_consumption,average_unit_gas_cost,gas_system_charges,gas_transport_cost,gas_material_cost,extra_fees
3,1.08,58.51,54.0,0.44,-0.88,17.66,23.68,0.02
4,0.86,383.66,447.0,0.32,14.71,63.59,141.57,0.00
5,0.86,386.63,448.0,0.37,16.97,44.45,167.54,-188.86
16,0.48,12.84,27.0,0.29,0.53,1.51,7.84,0.00
20,0.37,8.58,23.0,0.21,0.51,1.07,4.92,0.00
22,0.60,45.00,75.0,0.17,6.08,7.39,12.62,64.66
26,0.94,72.16,77.0,0.37,2.92,20.33,28.40,0.00
27,0.54,42.23,78.0,0.22,3.66,6.56,17.51,0.00
29,0.44,7.89,18.0,0.27,0.40,0.78,4.92,0.00
113,0.57,15.32,27.0,0.15,2.52,3.13,4.15,0.00


In [ ]:
myBase.cast_columns_types(mismatchedTypesCorrect)

,bill_id,F1_kWh,F2_kWh,F3_kWh,date,light_start_date,light_end_date,tv,gas_amount,gas_average_cost,light_average_cost,emission_date,supply_type,gas_start_date,gas_end_date,extra_fees,gas_consumption,light_consumption,gas_offer,light_offer_type,light_offer,howmuch_pay,total_amount,light_amount,average_unit_light_cost,average_light_bill_cost,average_unit_gas_cost,average_gas_bill_cost,billing_frequency,bill_type,gas_system_charges,light_system_charges,gas_material_cost,light_transport_cost,gas_transport_cost,light_material_cost
npartitions=80,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
,int64,float64,float64,float64,datetime64[ns],datetime64[ns],datetime64[ns],float64,float64,float64,float64,datetime64[ns],object,datetime64[ns],datetime64[ns],float64,float64,float64,float64,object,uint64,float64,float64,float64,float64,float64,float64,float64,string,bool,float64,float64,float64,float64,float64,float64
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [ ]:
#Questa colonna possiede solo questi 3 valori, converrebbe fare una one_hot_encoding o una mappatura dei valori 0:gas, 1:luce, 2:gas e luce (?)

myBase.df['supply_type'].drop_duplicates().compute()

0           gas
1          luce
2    gas e luce
Name: supply_type, dtype: object

In [ ]:
myBase.df.total_amount[myBase.df.total_amount == myBase.df.light_amount + myBase.df.gas_amount + myBase.df.extra_fees]

In [ ]:
# TO DO LIST

# fillna(0) sulle colonne dei costi
# localizzare outliers o con metodo grafico o con min e max
# condizioni sulle colonne
# analizzare bill_id e user_id per il merge
 
## RICEVIMENTO
# Costo non negativo
# con cosa decodificare
# sorting 
# come creare una reference
# average_gas_bill_cost ??